In [1]:
import pandas as pd
import csv
import requests
import json
# !pip3 install fuzzywuzzy
# !pip3 install fuzzysearch
# !pip3 install bs4
# !pip3 install python-Levenshtein
from fuzzysearch import find_near_matches
from bs4 import BeautifulSoup
import numpy as np
from fuzzywuzzy import process

In [2]:
with open('../data/raw/generate_data/x.txt') as json_file: 
    x = json.load(json_file)

In [3]:
a = []
hd = None
with open('../data/raw/generate_data/ArticleLinks.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:
        if hd is None:
          hd = row
          continue
        a.append({u: row[i] for i,u in enumerate(hd)})
a[:1]

[{'objectid': '1',
  'sector': 'Botanica',
  'time': '2/24/2018 1:18',
  'location': 'str. Melestiu 20/2',
  'source': ' ',
  'cars': '0',
  'trucks': '0',
  'motorcycle': '0',
  'bicycle': '0',
  'publictransport': '0',
  'pedestrian': '0',
  'other': '0',
  'placename': ' ',
  'photo': ' ',
  'video': ' ',
  'photo_link': '',
  'video_link': '',
  'day_night': '0',
  'raion': 'Chişinău',
  'comuna': 'Chişinău',
  'localitate': 'Chişinău',
  'year': '2018',
  'month': '12',
  'hour': '0',
  'weekday': '5',
  'day_part': 'noapte',
  'location_trasee': '',
  'street': 'Bulevardul Dacia',
  'tipul': 'Ciocnire fata in spate',
  'cauza': 'Nerespectarea distantei dintre vehicule',
  'decedati': '0',
  'traumati': '1'}]

In [4]:
def savepack(name,x, header):
  with open(name, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)
    for i in x:
      writer.writerow(i)

In [5]:
from nltk.stem.snowball import RomanianStemmer
st = RomanianStemmer()
sw = ['la','in','de', 'r', 'm','2']
def preprocesare(p):
  #return p
  diacritice = [['ș','s'],['ş','s'],['î','i'],['ă','a'],['ț','t'],['â','a'],['.',''],[',',''],['ţ','t']]
  schimb = [['str', 'strada'],['bd','bulevard'],['muncest','munc']]
  r = []
  p=p.replace('-',' ')
  for w in p.split(' '):
    w = w.strip().lower()
    
    if len(w)==0:
      continue
    #w = st.stem(w)
    for f,t in diacritice:
      w=w.replace(f,t)
    for f,t in schimb[2:]:
      w.replace(f,t)
    if w in sw:
      continue
    r.append(w)
  return ' '.join(r)

preprocesare('strada Decebal')

'strada decebal'

In [6]:
db = []
for v,i in x:
  o = {'id':i,'text':[preprocesare(v1) for v1 in v]}
  for p in ['localitate', 'location', 'street','pedestrian','decedati','sector','photo_link']:
    o[p] = preprocesare(a[i][p])
  db.append(o)

In [7]:
i=0
kw = ['tamponat', 'sofer', 'tampon', 'produs', 'avariat', 'accident','masin','ciocnit','incident','derap','impact','virar','intrat','traged','coliziun','lovit','mortal','pierdut','rasturnat']
verb = ['produs','apropiere','dat','intrat','fost','integistrat']
same = [['produs','avut loc','intamplat'],['masina','rutiera','autobuz','moticicleta'],['pieton','femeie','barbat','copil','victim','decedat','traumat'],
        ['accident','tragedie','coloziun','incident'],
        ['ciocnit','derapat','tamponat','derap','rasturnat']
        ]
bad = ['locuit','este din','originar din','locu']
sv = []
def checkprop(t):
      ww = [st.stem(w) for w in t.split(' ') if len(w)>0]
      #print(ww)
      ac = '0'
      for k in kw:
        if k in ww:
          ac='1'
      for k in bad:
        if k in ww:
          ac='0-'
      
      return ac
#savepack('../data/raw/generate_data/s.csv',sv,['id','text','y','com'])
checkprop('0 un sofer a tamponat o femeie pe trecerea pietoni intersectia bd decebal cu strada trandafirilor potrivit politiei persoana')

'1'

In [8]:
from fuzzywuzzy import fuzz
from sklearn.cluster import DBSCAN
dd = []
sv = []
nr=0
for id,d in enumerate(db[:900]):
  #text = ' '.join(d['text']).lower()
  #text.replace('str.','strada')
  #if intext(d['street'], text):
  needle = d['location'].lower().split(' ')
  pd = 160
  for text in d['text']:      
    if len(text)==0 or len(needle)==0 or needle is None:
      continue
    n1 = 0
    y = []
    fo = []
    ff = []
    ww = {}
    ln = len(needle)
    clustering = DBSCAN(eps=30, min_samples=max(2,len(needle)//2))
    for w in needle:
      if len(w)<=2 or w.isdigit():
        ln-=1
        continue
      m = find_near_matches(w, text, max_l_dist=len(w)//4)
      for n in m:
        dist = n.dist/len(w)
        if dist<0.4:
          n1+=1
          y.append([n.start])# and k<=n.end)
          fo.append(n.matched)
          ff.append(w)
          #break
    if len(y)==0:
        continue
    lb = clustering.fit(y).labels_
    ct = {}
    for l in lb:
        if l not in ct:
          ct[l]=0
        ct[l]+=1
    ct = sorted([m for m in ct.items()],key=lambda x:x[1],reverse=True)
    for c in ct:
        if c[0]!=-1 and c[1]>ln/2:
          #if n1>len(needle)/2:
          dd.append([i,y])
          nr+=1
          bw = [i[0] for j,i in enumerate(y) if lb[j]==c[0]]
          md = int(np.mean(bw))
          ty = ''
          tl = text
          ls = 0
          sl = []
          cc = {i:[] for i in needle}
          for j,i in enumerate(y):
             if lb[j]==c[0]:
               cc[ff[j]].append([j,i[0]])
          for i in cc.values():
            if len(i)>1:
              #print(i)
              mm = np.argmin([abs(j[1]-md) for j in i])
              for j,k in enumerate(i):
                if j!=mm:
                  #print(k[0])
                  lb[k[0]]=-1
          #mn = [i[0][1] for j,i in cc.items() if len(i)==1]
          #mn = 


          for j,i in enumerate(y):
             if lb[j]==c[0]:
               sl.append([i[0],fo[j]])
          sl = sorted(sl, key=lambda x:x[0])
          for i,f in sl:
            #print(f,ls, i)
            ty += tl[:i-ls]+'||'+f+"||"
            tl = tl[i-ls+len(f):]
            ls = i+len(f)
          ty+=tl
          
          #print(md, bw)
          sv.append([id, checkprop(text),  ty[max(0,md-pd):md+pd], ''])
          print('+',d['location'])
          print('-',' '.join([fo[j] for j in range(len(y)) if lb[j]==c[0]]))
          print(checkprop(text),'-',ty[max(0,md-pd):md+pd])
          print(','.join([str(i) for j,i in enumerate(y) if lb[j]==c[0]]))
          #break
      #break
    
nr, len(db)#, dd

+ bulevardul decebal strada trandafirilor
- decebal strada trandafirilor
1 - un sofer a tamponat o femeie pe trecerea pietoni intersectia bd ||decebal|| cu ||strada|| ||trandafirilor|| potrivit politiei persoana traumata a fost luata o ambulanta si transportata urgenta spital
[64],[75],[82]
+ traseul spre localitatea cheltuitori
- traseul spre localitatea cheltuitori
0 - aseara jurul orei 2330 politia capitalei a fost alertata despre inversarea unui automobil pe ||traseul|| ce duce ||spre|| ||localitatea|| ||cheltuitori|| soldat cu decesul unei persoane si altele trei traumatizate grav fata locului au intervenit grupa ope
[93],[109],[114],[126]
+ soseaua muncesti
- soseaua muncesti
1 - accidentul s a intamplat aceasta dimineata pe ||soseaua|| ||muncesti|| din capitala
[46],[54]
+ soseaua muncesti
- soseaua muncesti
1 - un pieton a fost spulberat mortal aseara un taxi accidentul s a produs pe ||soseaua|| ||muncesti|| din capitala jurul orei 21:00
[74],[82]
+ strada trandafirilor
- strad

+ traseul m3 orasul cimislia
- orasul cimislia
1 - un accident rutier a avut loc ||orasul|| ||cimislia|| aceasta dimineata urma impactului autotren ului si a unui automobil conducatoarea automobilului efectua virarea spre stanga pe banda dreapt
[30],[37]
+ strada alba iulia strada ion luca caragiale
- strada alba iulia uca
1 - un alt accident s a petrecut pe ||strada|| ||alba|| ||iulia|| din sectorul bui||uca||ni al capitalei din imagine se vede cum un microbuz model mercedes este grav accidentat
[32],[39],[44],[66]
+ traseul dintre satele isnovat si stetcani criuleni
- traseul dintre satele isnovat stetcani criuleni
1 - accidentul s a produs aceasta dupa amiaza pe ||traseul|| ||dintre|| ||satele|| ||isnovat|| si ||stetcani|| raionul ||criuleni||
[45],[53],[60],[67],[78],[95]
+ bulevardul dimitrie cantemir apropiere piata
- bulevardul dimitrie cantemir pita
1 - o fata 18 ani a fost lovita o masina timp ce traversa strada regulamentar accidentul a avut loc aseara pe ||bulevardul|| ||dim

+ satul baimaclia causeni
- satul baimaclia causeni
1 - accidentul s a intamplat aceasta noapte ||satul|| ||baimaclia|| raionul ||causeni|| potrivit oamenilor legii victima nu avea permis conducere
[40],[46],[64]
+ strada vadul lui voda strada mesterul manole
- strada vadul lui voda
1 - doua automobile s au ciocnit violent sectorul ciocana al capitalei accidentul s a produs pe ||strada|| ||vadul|| ||lui|| ||voda||
[92],[99],[105],[109]
+ strada banulescu bodoni strada 31 august 1989
- banulescu bodoni august
1 - doua masini model opel si toyota s au ciocnit frontal intersectia strazilor ||banulescu|| ||bodoni|| si 31 ||august|| 1989 din chisinau accidentul rutier a avut loc jurul orei 12:30 urma acestuia creandu se ambuteiaj zona
[76],[86],[99]
+ strada unirii strada 1 mai
- strada unirii mai
1 - accident grav straseni un copil care se afla sanius pe strada 1 ||mai|| spre ||strada|| ||unirii|| a intrat sub roata unei masini care se deplasa regulamentar pe strada unirii a orasului
[73],

+ strada calea basarabiei apropiere strada ismail
- strada calea basarabiei ismail
1 - un accident grav s a produs pe ||strada|| ||ismail|| intersectie cu ||calea|| ||basarabiei|| soferul unui camion a strivit un tanar care se deplasa pe o motoreta tanarul avea 22 ani si a murit pe loc din c
[31],[60],[66],[38]
+ strada ismail apropiere cu calea basarabiei
- ismail apropiere calea basarabiei
1 - pe str||ismail|| ||apropiere|| intersectie cu str||calea|| ||basarabiei|| este inregistrat accident rutier un motociclist s a tamponat intr o masina toyota fata locului a intervenit un e
[6],[13],[41],[47]
+ strada ismail apropiere cu calea basarabiei
- ismail calea basarabiei
0 - zona se circula cu dificultate pe str||ismail|| directia spre str||calea|| ||basarabiei|| astfel serviciul ”infotrafic” recomanda conducatorilor auto:
[37],[61],[67]
+ strada ismail pod
- strada ismail pod
1 - zapada si neatentia volan il va costa scump pe un sofer din capitala cu putin timp urma acesta a deraiat si s

(155, 379)

In [9]:
savepack('../data/raw/generate_data/v.csv',sv,['id','y','text','com'])

In [10]:
from fuzzywuzzy import fuzz
dd = []
nr=0
for i,d in enumerate(db[:10000]):
  #text = ' '.join(d['text']).lower()
  #text.replace('str.','strada')
  #if intext(d['street'], text):
  needle = d['street'].lower()  
  pd = 16
  for text in d['text']:      
    if len(text)==0 or len(needle)==0 or needle is None:
      continue  
    m = find_near_matches(needle, text.lower(), max_l_dist=6)
    ok=False
    nd = len(needle.split(' '))
    for n in m:
      dist = n.dist/len(needle)
      if dist<0.4:
        #print(m)
        k=0
        y = []
        sp = text.split(' ')
        for j,w in enumerate(sp):
          if k>=n.start and k<=n.end:
            y.append(j)
            if len(y)==nd:
              break
          k+=len(w)+1
        #print([sp[j] for j in y])
        dd.append([i,sp[max(y[0]-pd,0):y[0]+pd],y])
        nr+=1
        ok=True
        #mark matched substring as label - done
        #check keywords 0/1 accident location - done
        #check intersectie -> one more street then mark
        #sort by keywords
        #verify by hand
        #add negative example, outliers
        #make db with random changes
        #model basic to identify based on keywords and db of streeds/cities
        #rnn words/character train
        break
    if False and ok:
        break
nr, len(db)#, dd

(91, 379)

In [11]:
i=0
kw = ['tamponat', 'sofer', 'tampon', 'produs', 'avariat', 'accident','masin','ciocnit','incident','derap','impact','virar','intrat','traged','coliziun','lovit','mortal','pierdut','rasturnat']
verb = ['produs','apropiere','dat','intrat','fost','integistrat']
same = [['produs','avut loc','intamplat'],['masina','rutiera','autobuz','moticicleta'],['pieton','femeie','barbat','copil','victim','decedat','traumat'],
        ['accident','tragedie','coloziun','incident'],
        ['ciocnit','derapat','tamponat','derap','rasturnat']
        ]
bad = ['locuit','este din','originar din','locu']
sv = []
for id,t in enumerate(dd):
  h = 100
  ww=t[1]
  if True:
      ac = '0'
      for k in kw:
        if k in ww:
          ac='1'
      for k in bad:
        if k in ww:
          ac='0-'
      
      sv.append([str(id),' '.join(ww),'1',''])
      dd[id].append(ac)
      if len(ww)>0:
        w = ' '.join(ww)
        if len(w)<1000:
          i+=1
          ww=[]
          print(i,ac,w)
savepack('../data/raw/generate_data/s.csv',sv,['id','text','y','com'])

1 1 un sofer a tamponat o femeie pe trecerea pietoni intersectia bd decebal cu strada trandafirilor potrivit politiei persoana traumata a fost luata o ambulanta si transportata urgenta spital
2 0 accidentul s a intamplat aceasta dimineata pe soseaua muncesti din capitala
3 1 un pieton a fost spulberat mortal aseara un taxi accidentul s a produs pe soseaua muncesti din capitala jurul orei 21:00
4 0 accidentul a avut loc sectorul botanica pe strada trandafirilor jurul orei 20:00
5 1 potrivit ofiterului presa al politiei capitalei natalia stati accidentul a avut loc jurul orei 11 pe strada valea crucii doua automobile unul mazda s a tamponat frontal cu un automobil model toyota
6 1 doua masini s au ciocnit aseara intersectia bulevardelor dacia si traian din capitala urma impactului un taxi s a rasturnat iar celalalt automobil
7 1 accident lant pe strada muncesti din capitala impactul s a produs cu putin timp urma
8 1 accident rutier intersectia strazilor columna – maria cebotari dupa ce u

In [12]:
stp = ['şoseaua']
nr = 0
w=0
def intext(d, text):
  #return text.find(d)!=-1
  return text.find(d.replace('-',' ').replace('muncest','munc').replace('soseau','').replace('bd.','').lower().replace('str.','').replace('strad','').replace('soseaua','').replace('Şoseaua','').replace('  ',' ').replace('bulevard','').strip())!=-1
    
win = {}
wout = {}
wst = {}
x = []
y = []
stt = []
w1=0
for d in db:
  text = ','.join(d['text']).lower()
  if  len(d['street'])==0 or d['street'] is None:
      continue  
  #text.replace('str.','strada')
  if intext(d['street'], text):
    stt.append(d['street'])
    for w in d['street'].split(' '):
          if w not in wst:
            wst[w]=0
          wst[w]+=1
    nr+=1
    for t in d['text']:
      x.append(t)
      if intext(d['street'], t):
        y.append(1)
        for w in t.split(' '):
          if w not in win:
            win[w]=0
          win[w]+=1
      else:
        y.append(0)
        for w in t.split(' '):
          if w not in wout:
            wout[w]=0
          wout[w]+=1
  else:
    if 0:
      print(d['text'])
      print(d['street'],',', d['location'])
      print(d['photo_link'])
      w1+=1
      if w1>18:
        break
import numpy as np
nr, len(db), len(win), len(x), np.sum(y)

(36, 379, 464, 527, 39)

In [44]:
keyword  = ['intersectia','strada','bulevard','sosea','sat','oras','raion','capitala','sector']
st = RomanianStemmer()
for q in keyword:
  w = st.stem(q)
  print('in:',w,win[w] if w in win else '-')
  print('out',w,wout[w] if w in wout else '-')

in: intersect -
out intersect -
in: strad -
out strad -
in: bulevard -
out bulevard -
in: sos -
out sos -
in: sat -
out sat -
in: oras -
out oras 1
in: raion -
out raion -
in: capital -
out capital -
in: sector -
out sector -


In [14]:
i=0
kw = ['tamponat', 'sofer', 'produs', 'accident','masin','ciocnit','incident','derap','impact','virar','intrat','traged','coliziun','lovit','mortal','pierdut','rasturnat']
verb = ['produs','apropiere','dat','intrat','fost','integistrat']
same = [['produs','avut loc','intamplat'],['masina','rutiera','autobuz','moticicleta'],['pieton','femeie','barbat','copil','victim','decedat','traumat'],
        ['accident','tragedie','coloziun','incident'],
        ['ciocnit','derapat','tamponat','derap','rasturnat']
        ]
bad = ['locuit','este din','originar din','locu']
sv = []
for id,t in enumerate(x):
  h = 100
  www=t.split(' ')
  ww = []
  for j,w in enumerate(www):
    if w == "intersect":
      ww = www[max(0,j-16):min(len(www),j+10)]
      ac = '0'
      for k in kw:
        if k in ww:
          ac='1'
      for k in bad:
        if k in ww:
          ac='0-'
      
      sv.append([str(id),' '.join(ww),'1',''])
      if len(ww)>0:
        w = ' '.join(ww)
        if len(w)<1000:
          i+=1
          ww=[]
          print(ac,i,w)
savepack('../data/raw/generate_data/s.csv',sv,['id','text','y','com'])

In [15]:
swtl = sorted([[i,j] for i,j in wst.items()], key=lambda x: x[1], reverse=True)
swtl

[['strada', 32],
 ['calea', 4],
 ['ismail', 4],
 ['iesilor', 3],
 ['trandafirilor', 2],
 ['mihai', 2],
 ['soseaua', 2],
 ['balcani', 2],
 ['ion', 2],
 ['bulevardul', 2],
 ['miorita', 2],
 ['valea', 1],
 ['crucii', 1],
 ['maria', 1],
 ['cebotari', 1],
 ['viteazul', 1],
 ['pelivan', 1],
 ['vasile', 1],
 ['alecsandri', 1],
 ['alba', 1],
 ['iulia', 1],
 ['dimitrie', 1],
 ['cantemir', 1],
 ['kiev', 1],
 ['doina', 1],
 ['1', 1],
 ['mai', 1],
 ['dimo', 1],
 ['alecu', 1],
 ['russo', 1],
 ['gheorghe', 1],
 ['asachi', 1],
 ['ciuflea', 1],
 ['stefan', 1],
 ['cel', 1],
 ['mare', 1],
 ['costiujeni', 1],
 ['alexei', 1],
 ['mateevici', 1],
 ['dacia', 1],
 ['creanga', 1],
 ['uzinelor', 1],
 ['orheiului', 1],
 ['vadul', 1],
 ['lui', 1],
 ['voda', 1],
 ['eminescu', 1]]

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(x)
X_train_counts.shape


(527, 2102)

In [17]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(527, 2102)

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, y, test_size=0.33, random_state=42)

In [19]:
from sklearn.naive_bayes import MultinomialNB
import numpy as np
clf = MultinomialNB().fit(X_train, y_train)
predicted = clf.predict(X_test)
np.mean(predicted == y_test)

0.9022988505747126

In [20]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42).fit(X_train, y_train)
predicted = clf.predict(X_test)
np.mean(predicted == y_test)

0.9655172413793104

In [21]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                    ('clf', MultinomialNB()),
 ])

In [22]:
from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 2)],
            
               'clf__alpha': (1e-2, 1e-3) }
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [23]:
gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)
gs_clf = gs_clf.fit(X_train, y_train)
predicted = gs_clf.predict(X_test)
np.mean(predicted == y_test)

0.9425287356321839

In [24]:
winl = sorted([[i,j] for i,j in win.items()], key=lambda x: x[1], reverse=True)
woutl = sorted([[i,j] for i,j in wout.items()], key=lambda x: x[1], reverse=True)
woutl[:30], winl[:30]

([['a', 194],
  ['', 133],
  ['si', 106],
  ['cu', 73],
  ['sa', 70],
  ['au', 62],
  ['o', 61],
  ['un', 61],
  ['pe', 55],
  ['ca', 55],
  ['fost', 54],
  ['se', 52],
  ['{', 50],
  ['din', 47],
  ['=', 47],
  ['nu', 46],
  ['s', 41],
  ['pentru', 40],
  ['/', 36],
  ['}', 34],
  ['(video)', 28],
  ['2020', 28],
  ['soferul', 27],
  ['mai', 27],
  ['care', 27],
  ['accident', 26],
  ['iar', 23],
  ['var', 23],
  ['ce', 22],
  ['masini', 21]],
 [['a', 76],
  ['pe', 45],
  ['un', 38],
  ['strada', 38],
  ['din', 27],
  ['s', 23],
  ['cu', 21],
  ['o', 16],
  ['avut', 16],
  ['accident', 16],
  ['fost', 15],
  ['loc', 14],
  ['accidentul', 13],
  ['capitala', 13],
  ['automobil', 10],
  ['se', 10],
  ['dimineata', 9],
  ['au', 8],
  ['aceasta', 8],
  ['ce', 8],
  ['model', 7],
  ['produs', 7],
  ['care', 7],
  ['unui', 7],
  ['ani', 7],
  ['intersectia', 6],
  ['si', 6],
  ['sectorul', 6],
  ['urma', 6],
  ['rutier', 6]])

In [25]:
b = []
hd = None
with open('../data/raw/generate_data/text_db.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    
    for row in readCSV:
        if hd is None:
          hd = row
          continue
        b.append({u: row[i] for i,u in enumerate(hd)})
b[:1]

[]

In [26]:
hasT = [f for f in a if f['photo_link'].find('.jpg')!=-1]
len(hasT)

112

In [27]:
[f['photo_link'] for f in hasT[:5]]

['https://i.imgur.com/tcz3Qxr.jpg?1',
 'https://media.publika.md/md/image/201902/w720/accident_94453600.jpg',
 'https://media.publika.md/md/image/201902/w720/2c21c013eb758f9e5967843c3c09f36dd26ca732_36746400.jpg',
 'https://media.publika.md/md/image/201902/w720/51429086_648554888934620_3010909367420583936_n_08031000.jpg',
 'https://i.imgur.com/TjzDy6r.jpg']

In [28]:
def getText(url):
      try:
        page = requests.get(url)
      except:
        return None
      soup = BeautifulSoup(page.content, 'html.parser')
      return soup

soup = getText(hasT[2]['photo_link'])
[p.get_text() for p in soup.find_all('p') ]

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


[]

In [29]:
x = []
for i,l in enumerate(a):
  if l['photo_link'].find('http')==-1 or len(l['photo_link'].strip())==0:
    continue
  soup = getText(l['photo_link'])
  if soup is None:
    continue
  p = [p.get_text() for p in soup.find_all('p') ]
  if len(p)>0:
    x.append([p,i])
len(x)

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHA

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHA

353

In [30]:
len(x)

353

In [31]:
x[0]

[['canal tv',
  '\xa0',
  'Un şofer a tamponat\xa0o femeie\xa0pe trecerea de pietoni, la intersecţia bd. Decebal cu strada Trandafirilor.\xa0Potrivit poliţiei, persoana traumată a fost luată de o ambulanţă şi transportată de urgenţă la spital.\xa0',
  'Alte circumstanţe ale accidentului se stabilesc.\xa0',
  'Dacă ai fost martorul unei situaţii interesante, expediază imaginile surprinse pe adresa\xa0contact@publika.md\xa0sau prin intermediul opţiunii\xa0ŞTIREA TA, de pe site-ul\xa0PUBLIKA.MD, iar acestea vor fi văzute de o țară întreagă.\xa0',
  'Totodată, puteți să ne trimiteți materiale și pe\xa0\xa0sau\xa0\xa0Whatsapp la telefonul\xa0060 88 11 00.'],
 1]

In [32]:
with open('../data/raw/generate_data/text_db.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["id","text"])
    for v,i in x:
      writer.writerow([str(i),'###'.join(v)])
      
'done'

'done'

In [40]:

 with open('../data/raw/generate_data/x.txt', 'w') as outfile:
          json.dump(x, outfile)

In [41]:
with open('../data/raw/generate_data/x.txt') as json_file:
    x = json.load(json_file)
x[0]

[['canal tv',
  '\xa0',
  'Un şofer a tamponat\xa0o femeie\xa0pe trecerea de pietoni, la intersecţia bd. Decebal cu strada Trandafirilor.\xa0Potrivit poliţiei, persoana traumată a fost luată de o ambulanţă şi transportată de urgenţă la spital.\xa0',
  'Alte circumstanţe ale accidentului se stabilesc.\xa0',
  'Dacă ai fost martorul unei situaţii interesante, expediază imaginile surprinse pe adresa\xa0contact@publika.md\xa0sau prin intermediul opţiunii\xa0ŞTIREA TA, de pe site-ul\xa0PUBLIKA.MD, iar acestea vor fi văzute de o țară întreagă.\xa0',
  'Totodată, puteți să ne trimiteți materiale și pe\xa0\xa0sau\xa0\xa0Whatsapp la telefonul\xa0060 88 11 00.'],
 1]

In [42]:
import random
vector = range(1,10)# [1,2,3,4,5,6,7,8,9]
exista = {}
for i in range(10000):
  numere = []
  numar = 1
  for index in range(1,9):
    semn = random.choice(['+','-','nimic'])
    if semn == 'nimic':
      numar

In [43]:
rrl = []
with open('../data/raw/generate_data/v.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["id","text"])
    for v,i in x:
      writer.writerow([str(i),'###'.join(v)])
      